In [1]:
import simpy
from scipy.stats import expon
from scipy.stats import norm

In [2]:
class GasStation():
    
    def __init__(self, env, number_of_pumps):
        
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=number_of_pumps)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Attributes:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump

    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_station,
        env
    ):
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_station = gas_station
        self.env = env
        
        # start the process in the sim
        self.action = self.env.process(self.run())
        
    def run(self):
        
        print(f'Car {self.id} arrives at {self.env.now} minutes')
                
        with self.gas_station.resource.request() as req:
            
            yield req
            
            print(f'Car {self.id} begins utilizing a pump at {self.env.now} minutes')
            
            pump_time = self.gas_required / PUMP_RATE
            
            utilization = max(pump_time, self.lay_time)
            print(utilization)
            
            yield self.env.timeout(utilization)
            
            print(f'Car {self.id} leaves its pump at {self.env.now} minutes')
            
            self.gas_station.minutes_utilized += utilization
            self.gas_station.gallons_sold += self.gas_required

In [4]:
def scheduler(env):
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=EXPECTED_WAITING_TIME)
        
        std_norm = norm.rvs()
        gas_required = std_norm * GAS_REQUIRED_STD + GAS_REQUIRED_MEAN
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=LAY_TIME_MEAN)
        
        yield env.timeout(waiting_time)
        
        Car(
            id=id,
            gas_required=gas_required,
            lay_time=lay_time,
            gas_station=gas_station,
            env=env
        )
        
        id += 1

In [5]:
PUMP_RATE = 10 # in gallons per minute

# waiting time between cars is exponentially distributed
EXPECTED_WAITING_TIME = 2 # the time between arrivals in minutes

# gas required is normally distributed
GAS_REQUIRED_MEAN = 15 # in gallons
GAS_REQUIRED_STD = 5 # in gallons

# lay time is exponentially distributed
LAY_TIME_MEAN = 5 # in minutes

In [6]:
env = simpy.Environment()
gas_station = GasStation(env, 1)
env.process(scheduler(env))
env.run(until=100)

Car 0 arrives at 0.856077913588674 minutes
Car 0 begins utilizing a pump at 0.856077913588674 minutes
4.767260077656789
Car 1 arrives at 2.256424294088653 minutes
Car 2 arrives at 2.7559740794398295 minutes
Car 3 arrives at 4.023805766187032 minutes
Car 0 leaves its pump at 5.623337991245463 minutes
Car 1 begins utilizing a pump at 5.623337991245463 minutes
5.859481175102451
Car 4 arrives at 6.4342755833567224 minutes
Car 5 arrives at 9.923108257053393 minutes
Car 6 arrives at 10.747416139445606 minutes
Car 7 arrives at 11.300721479420899 minutes
Car 1 leaves its pump at 11.482819166347914 minutes
Car 2 begins utilizing a pump at 11.482819166347914 minutes
3.7539232327868444
Car 8 arrives at 11.989195699897635 minutes
Car 9 arrives at 13.421068895087597 minutes
Car 10 arrives at 14.308675281547686 minutes
Car 11 arrives at 14.508388967384718 minutes
Car 2 leaves its pump at 15.236742399134759 minutes
Car 3 begins utilizing a pump at 15.236742399134759 minutes
0.6130223025364467
Car 3 l